In [3]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
file_path = r'C:\Users\Oh\makedata.csv'
train = pd.read_csv(file_path)
train.head()

,Q,A,id
0,오늘 저녁에 오랜만에 영화 고고?,오우 너무 좋지? ㅎ,1
1,몇 시에 퇴근해?,난 3시면 퇴근하지요 ㅎ 자기는 ?,2
2,나는 5시 인데 회사 일이 많아서 조금 더 늦을 수 있어,그럼 영화관 가기는 그렇고 집에서 볼까?,3
3,좋지 좋지 ㅎ 집에서 치맥 하면서 볼까?,너무 좋지 하하 상상만 해도 벌써 힐링인데 ㅎ,4
4,근데 자기 다이어트 한다며 ㅋ 괜찮겠어?,맛있으면 0칼로리라잖아 ㅎ,5


In [11]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import numpy as np
from tqdm import tqdm

# CSV 파일을 읽기
file_path = r'C:\Users\Oh\makedata.csv'
train = pd.read_csv(file_path)
train = train[0:10]
# 'translated' 열 초기화
train['translated'] = [''] * len(train)

# WebDriver 설정 및 초기화
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=options)

src, dest = 'ko', 'en'
translate_url = f'https://papago.naver.com/?sk={src}&tk={dest}'

translated, errors = list(), list()

driver.get(translate_url)
time.sleep(3)  # 페이지 로딩 대기

# 리뷰 번역 루프
for i, review in enumerate(tqdm(train['Q'])):
    try:
        driver.find_element(By.ID, 'txtSource').clear()
        driver.find_element(By.ID, 'txtSource').send_keys(review)
        time.sleep(np.sqrt(np.log(len(review))) + 1)  # 입력 대기 시간
        translated_text = driver.find_element(By.ID, 'txtTarget').text
        train['translated'][i] = translated_text
    except Exception as e:
        print(f"Error at index {i}: {e}")
        errors.append(i)

driver.quit()

# 번역된 데이터를 새로운 CSV 파일로 저장
output_file_path = r'C:\Users\Oh\makedata_translated.csv'
train.to_csv(output_file_path, index=False)


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]C:\Users\OH\AppData\Local\Temp\ipykernel_19860\4165241607.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['translated'][i] = translated_text
 10%|████████▎                                                                          | 1/10 [00:02<00:25,  2.82s/it]C:\Users\OH\AppData\Local\Temp\ipykernel_19860\4165241607.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['translated'][i] = translated_text
 20%|████████████████▌                                                              

In [14]:
pip install nlpaug

   ---------------------------------------- 0.0/410.5 kB ? eta -:--:--
   ----- ---------------------------------- 61.4/410.5 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------  409.6/410.5 kB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 410.5/410.5 kB 4.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import pandas as pd
import nlpaug.augmenter.word as naw

# CSV 파일을 읽기
file_path = r'C:\Users\Oh\makedata_translated.csv'
train = pd.read_csv(file_path)

# 데이터 구조 확인
print(train.head())
print(train.columns)

# 데이터 증식기를 설정
aug = naw.SynonymAug(aug_src='wordnet')

# 번역된 텍스트 데이터 증식
train['augmented'] = train['translated'].apply(lambda x: aug.augment(x))

# 증식된 데이터를 새로운 CSV 파일로 저장
output_file_path = r'C:\Users\Oh\makedata_augmented.csv'
train.to_csv(output_file_path, index=False)



                                 Q                          A  id  \
0               오늘 저녁에 오랜만에 영화 고고?                오우 너무 좋지? ㅎ   1   
1                        몇 시에 퇴근해?        난 3시면 퇴근하지요 ㅎ 자기는 ?   2   
2  나는 5시 인데 회사 일이 많아서 조금 더 늦을 수 있어     그럼 영화관 가기는 그렇고 집에서 볼까?   3   
3           좋지 좋지 ㅎ 집에서 치맥 하면서 볼까?  너무 좋지 하하 상상만 해도 벌써 힐링인데 ㅎ   4   
4           근데 자기 다이어트 한다며 ㅋ 괜찮겠어?             맛있으면 0칼로리라잖아 ㅎ   5   

                                          translated  
0  It's been a while since I watched a movie this...  
1                     What time do you get off work?  
2  It's 5 o'clock, but I have a lot of work to do...  
3  Sounds good. Should we watch it at home while ...  
4       But you said you're on a diet. Are you okay?  
Index(['Q', 'A', 'id', 'translated'], dtype='object')


In [19]:
pd.read_csv(output_file_path)

,Q,A,id,translated,augmented
0,오늘 저녁에 오랜만에 영화 고고?,오우 너무 좋지? ㅎ,1,It's been a while since I watched a movie this...,"[""It ' s been a patch since I watch a film thi..."
1,몇 시에 퇴근해?,난 3시면 퇴근하지요 ㅎ 자기는 ?,2,What time do you get off work?,['What time answer you get slay work?']
2,나는 5시 인데 회사 일이 많아서 조금 더 늦을 수 있어,그럼 영화관 가기는 그렇고 집에서 볼까?,3,"It's 5 o'clock, but I have a lot of work to do...","[""Information technology ' s 5 atomic number 8..."
3,좋지 좋지 ㅎ 집에서 치맥 하면서 볼까?,너무 좋지 하하 상상만 해도 벌써 힐링인데 ㅎ,4,Sounds good. Should we watch it at home while ...,['Strait salutary. Should we watch it at plate...
4,근데 자기 다이어트 한다며 ㅋ 괜찮겠어?,맛있으면 0칼로리라잖아 ㅎ,5,But you said you're on a diet. Are you okay?,"[""Merely you say you ' re on a dieting. Are yo..."
5,영화엔 카라멜 팝콘도 필수지,카라멜 팝콘은 어디서 사려구 ?,6,Caramel popcorn is a must for movies,['Caramel popcorn be a moldiness for movies']
6,집에 가는 길에 마트 들려서 사지 뭐 ㅎ,좋아 그럼 자기가 집에 가는 길에 좀 사서 가줘,7,I'll stop by the mart on my way home,"[""Ane ' ll stop by the marketplace on my way h..."
7,응응 내가 사서 갈게,콜라도 필요하겠지?,8,"Yes, I'll buy it and go","[""Yes, I ' ll corrupt it and move""]"
8,당연하지 하하 오늘은 홈 영화관은 오픈 하는 것이네 ㅎ,옹 내가 다 꾸며 놓겠어 기대하시라,9,Of course haha today the home movie theater is...,['Of course haha today the abode movie theater...
9,기대하고 퇴근하겠어 ㅎ,보고 싶은 영화 골라나,10,I'll look forward to going home,"[""I ' ll wait forward to going house""]"
